### Importacion de librerias

In [31]:
import os # Sistema operativo
from PIL import Image # Carpeta de imagenes
import numpy as np # Matrix con las imagenes
import cv2 # OpenCV
import pickle # Guardar modelo

### Importar haarcascade y carpetas

In [32]:
# Se llamar archivo cascada para caras frontales sirve en caso de frente a pantalla
#FrontFace = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_alt2.xml')
FrontFace = cv2.CascadeClassifier('HaarCascade/haarcascade_frontalface_default.xml')
reconocer_cara = cv2.face.LBPHFaceRecognizer_create() # LBPH -> patrones para reconocer caras

# Recolectar imagenes
path_image = os.path.join("caras") #contiene las imagenes de las distintas carpetas

### Creacion de variable

In [33]:
entrenamiento = [] # Contiene las imagenes
etiqueta = [] # Es el nombre de las carpetas que contiene imagenes.
id_actual = 0 # Mantiene el id de la carpeta actual
id_etiqueta = {} # Contendra nombre de las etiquetas

### Se recorre carpeta para poder hacer respectivos trabajos sobre ello para posteriormente hacer trabajos

In [34]:
for root, directorio, archivos in os.walk(path_image):
    for archivo in archivos:
        if archivo.endswith("jpg") or archivo.endswith("png"): # Tipo de archivo soportados
            path = os.path.join(root, archivo) # Direccion archivo en carpeta
            obtener_etiqueta = os.path.basename(os.path.dirname(path)).lower() # Nombre de las carpetas, sirve para las etiquetas
            #print (path, obtener_etiqueta)

            if not obtener_etiqueta in id_etiqueta: # si no esta etiqueta en id_etiqueta
                id_etiqueta[obtener_etiqueta] = id_actual # se agrega la etiqueta al grupo de esta
                id_actual += 1
            id_momento = id_etiqueta[obtener_etiqueta] # se etiqueta segun si pertenece o no
            #print(id_etiqueta)

            PIL_image = Image.open(path).convert("L") # Se cargan imagenes y se pasan a escala de grises
            Array_image = np.array(PIL_image, "uint8") # Imagenes pasan a array y quedan en entero de 0 a 255

            caras = FrontFace.detectMultiScale(Array_image, scaleFactor=1.5, minNeighbors=9) # Vecinos al rectacgulo en el que se esta

            # x, y, ancho, alto -> 0, 0, x+n, y+n
            for (x, y, w, h) in caras:
                obtener_cara = Array_image[y:y+h, x:x+w] # Se obtiene un cuadro con el tamaño de la cara
                entrenamiento.append(obtener_cara) # se agrega en entrenamiento la cara
                etiqueta.append(id_momento) # se agrega la etiqueta a la que pertenece la cara

### Se crea archivo con etiquetas

In [35]:
with open("labels.pickle", "wb") as f:
    pickle.dump(id_etiqueta, f) # se guarda con jerarquia el modelo de etiqueta

### Se crea entrena y se guarda modelo

In [36]:
reconocer_cara.train(entrenamiento, np.array(etiqueta)) # se entrena modelo imagen vs etiqueta
reconocer_cara.save("entrenamiento.yml") # se guarda modelo entrenado.